# Notebook to introduce respy 
Introduces basic functions of library for processing respiration recordings (single best chest stretch).

This library provides some basic signal processing tools to extract respiration phase and related features from recordings of respiration from healthy awake adults. Initial developed to track respiration phase in music listeners (2019), the features are now being adapted to also detect respiration phase in performing musicians (2023) and similar active conditions. 

Chest stretch measurements do not perfectly capture airflow. Many interesting aspects of respiratory behaviour require more demanding sensor arrangements to accurately capture gas partial pressures and volumns of air displaced. While measured change in chest circumference is often analogous to air flow, changes in posture and respiration strategy can shift the scale. 

This library instead focuses on respiratory phase timing, which is more reliably captured with this sensor. (Though there is also some noise here as well.) This time domain analysis of a quasi-oscillatory signal uses optimised heuristics to distinguish respiratory events from sensor noise. With accurate detection of indiviual breaths, many interesting follow questions can be asked, such as the relationship of respiratory phase to concurrently presented stimuli or bodily actions. 

And after respiratory phase information has been extracted with some degree of reliability, respiratory shape and style can be assessed with much greater refinement. This package allows for the preliminary classification of respiratory activity into:

1. Quiet breathing 
2. Augemented breaths
3. Vocalisation, speach and song
4. Wind instrument playing (open and resistance pressure)
5. Elevated respiration (physical exertion)
6. Free respiration (not quiet, not exertion)

This notebook uses a set of demo respiraton recordings to demonstrate the phase extraction and classification of respiratory behaviour.

# Respiration phases

Human respiration can usually be broken down into three intervals: Inspiration phase, expiration phase, and post-expiration pause. For vocalisation, diving, and other distinct conditions, additional pauses can be injected within the Inspiration-Expiration interval. 

### Inspiration
The inspiration phase of human respiration is always active: respiratory muscles coordinate to expand lung volumn, creating negative pressure to draw fresh air into the body. One of the most important muscles for this process is the diapham, a wide stretch of muscle between the chest and abdominal cavity that curves up when relaxed and lowers/flattens with contraction to expand the volumn of lung tissue. Other key muscles span the rib cage (obliques), positioned to lift the bones and expand this structure when contracted. (It is this expansion that is measured by single-belt chest stretch sensors.) Inspirations can also be deepened and extended with cooperation from the abdominals, trapezius, etc. A wonder and challenge for capturing the details of inspiration is the flexibility of this phase. There are many ways to produce the same displacement of air in the same ammount of time, and the specific inspiration action performed depends on posture, practice, and sensory conditions on top of the functional priority of gas exchange. When deliberately extended, an inspiration phase can last several seconds, but usually breath is actively being drawn for only 400 to 1100 ms. Closer to a second for quiet breathing, less than half that when inspiring for speech or other actions. 

### Expiration
The expiration phase of the human respiratory cycles can active or entirely elastic. Under relaxed conditions, expiration is usually passive and mechanical. Instead of a network of muscles working to compress the lungs, the respiratory system is simply allowed to revert to a stable state through the natural relaxation of the inspiration muscles and elastic contraction of lung tissue. During quiet breathing, sleep, and similar states, elastic expiration directly follows the peak of inspiration, releasing the potential energy injected into the system by the preceeding actively drawn of breath. 

Active expiration can take many forms. The expiration muscles serve to contract the rib cage (intercostals) and push up on the diaphram (abdominals), forcing air out of the lungs more quickly and past the point of relaxed stability if greater volumns of air need to be displaced per cycle. Physical exersion often requires faster gas exchange than can be achieved with elastic expiration, and the result is a cycle that can carry momentum between phases in both directions, potential energy also aiding the transition from expiration to inspiration. This mechanical component of this respiratory pattern results in a shape of chest stretch measurements closer to the familiar sinusoide as the system is tuned to meet a steady high drive for CO2 expulsion.

Vocalisation and other expressive expirations are also active, though their shape can be quite different from expiration for exertion. Exquisitely controlled lung pressure for sound production is achieved through the cooperative engagement of both inspiration and expiration muscles. With some practice, humans can extend the expiration phase from a smooth 1.5-2 second exhale to a steady outflow of air over tens of seconds. And expirations for vocalisation are not always smooth, they are often paused for articulation and interrupted for mini inspirations. 

In chest stretch measurements, the shape of this kind of active exhale depends in part on whether the sound generation involves some form of blockage or resistence. For a high pressure instrument or sharp yell, the chest cavity can be compressed to increase the pressure before any air is released. Additionally, these breaths often involve displacing extraordinary volumns of air per cycle and a single chest stretch sensor can not be counted on to capture all that is happening to support these extended expertly controlled exhalations. As such, the respy library only gives a best guess at the onset, end, and character of expressive expirations from the signal information available. 

### Post-expiration pause
When demand for air flow is normal and low, the default state of the lungs is mostly deflated. Active inspiration flows to elastic expiration and then the system waits to be prompted to inflate again. This interval of waiting is the post expiraton pause, and it is the most variable phase of respiration in stable conditions. It is also what breaks oscillatory models of respiration outside of high exersion. The respiratory system has no memory: after air is expelled in expiration, there is no mechanical drive to keep breathing the same way as before. Instead, the respiratory system seems oriented towards coordinating between current demand and upcoming actions. Flexibility in inspiration depth makes it easy to adjust to small delays or advances in inspiration time without risking homeostasis. As such, the post expiration pause can be less than 400 ms one cycle and over 2000 ms the next without ever disturbing the breather. # check these numbers empirically

During active exertion, of course, there is no post-expiration pause at all. And during expressive expirations like speech, there can be pauses that are not post-expiration. However, the sensory and physiological conditions of the post-expiration pause make it well worth tracking when considering respiratory phase timing in really everyday respiratory sequences. 

### Other Pauses

As the respy library is extended to capture phase information in respiration sequences during more varied forms of action, it is useful to identify more effortful pauses in the respiratory cycle. These can occure in the preparation of sound generating expulsions or as some measurable duration of stable chest circumpherence between intervals of slow contraction. This isn't part of respy yet, but it needs to be. That's why we are tryinging the algorithm a new with more varied measurements. 

# Loading and evaluating respiration features

Examples from 

http://localhost:8888/notebooks/Audience_Resp/Heaphones_Speakers/Jupiter_resp/ICMPC-respAttentive.ipynb

# Evaluating phase alignment in repeated listenings

http://localhost:8888/notebooks/Audience_Resp/Heaphones_Speakers/Jupiter_resp/Repeated%20Response%20Resp%20Phases.ipynb

# Evaluating respiratory alignment across players (repeated)

http://localhost:8888/notebooks/Stavanger/Coordination/Resp_Wrist_coordination.ipynb


